## Metrics Calculations Tutorial

This notebook provides examples on how to carry out data metrics calcuations and analysis using the post_processing python library. Be sure to go through the [Quick Start](https://nhs-postprocessing.readthedocs.io/en/stable/QuickStart.html) section of the [documentation](https://nhs-postprocessing.readthedocs.io/en/stable/index.html) for instructions on how to access and import the libary and its packages.

If you would like to open an editable runnable version of the tutorial click [here](https://mybinder.org/v2/gh/UchechukwuUdenze/NHS_PostProcessing/main?%2FHEAD=&urlpath=%2Fdoc%2Ftree%2Fdocs%2Fsource%2Fnotebooks%2Ftutorial-metrics.ipynb) to be directed to a binder platform

<mark>The Library is still under active development and empty sections will be completed in Due time</mark>

### Table of content
- [Available Metrics](#available-metrics)
- [Single Data Metrics](#single-data-metrics)
- [Comparison Metrics](#comparison-metrics)

 All files are available in the github repository [here](https://github.com/UchechukwuUdenze/NHS_PostProcessing/tree/main/docs/source/notebooks)

### Requirements

The conda environmnent contains all libraries associated the post processing library. After setting up the conda environment, you only have to import the metrics maniupulation module from postprocessinglib.evaluation.

In [1]:
### Remove and modify these later.
import sys
import pandas as pd
sys.path.append("../../../")

In [2]:
from postprocessinglib.evaluation import data, metrics

Lets use one of the data blocks from the data manipulation tutorial

In [3]:
# passing a controlled csv file for testing
path_output = "MESH_output_streamflow_3.csv"
path_input = "Station_data.xlsx"

DATAFRAMES = data.generate_dataframes(csv_fpaths=path_output, warm_up=91)
               
Stations = pd.read_excel(io=path_input)

ignore = []
for i in range(0, len(Stations)):
    if Stations['Properties'][i] == 'X':
        ignore.append(i)

Stations = Stations.drop(Stations[Stations['Properties'] == 'X'].index)
Stations = Stations.set_index('Station Number')

for i in reversed(ignore):
        DATAFRAMES["DF_OBSERVED"] = DATAFRAMES["DF_OBSERVED"].drop(columns = DATAFRAMES['DF_OBSERVED'].columns[i])
        DATAFRAMES['DF_SIMULATED']  = DATAFRAMES["DF_SIMULATED"].drop(columns = DATAFRAMES['DF_SIMULATED'].columns[i])
        for key, dataframe in DATAFRAMES.items():
            if key != "DF_SIMULATED" and key != "DF_OBSERVED" and key != "NUM_SIMS":
                DATAFRAMES[key] = dataframe.drop(columns = dataframe.columns[[2*i, 2*i+1]])
            

# for key, value in DATAFRAMES.items():
#     print(f"{key}:\n{value.head}")

The start date for the Data is 1982-01-01


Now that we have our data, let's jump right in!

### Available Metrics

Because the library is in active development, there will be regular removals and additions to its features. As a rule of thumb therefore it is always a good idea to check what it can do at the time of use. We can do this by going ->

In [4]:
metrics.available_metrics()

['MSE - Mean Square Error',
 'RMSE - Roor Mean Square Error',
 'MAE - Mean Average Error',
 'NSE - Nash-Sutcliffe Efficiency ',
 'NegNSE - Nash-Sutcliffe Efficiency * -1',
 'LogNSE - Log of Nash-Sutcliffe Efficiency',
 'NegLogNSE - Log of Nash-Sutcliffe Efficiency * -1',
 'KGE - Kling-Gupta Efficiency',
 'NegKGE - Kling-Gupta Efficiency * -1',
 'KGE 2012 - Kling-Gupta Efficiency modified as of 2012',
 'BIAS- Prcentage Bias',
 'AbsBIAS - Absolute Value of the Percentage Bias',
 'TTP - Time to Peak',
 'TTCoM - Time to Centre of Mass',
 'SPOD - Spring Pulse ONset Delay',
 'FDC Slope - Slope of the Flow Duration Curve']

### Single Data Metrics
These are the metrics that only apply to just one of either the simulated or observed data. They are less about analysis and more about obtaining information about the data. These aren't made to compare but rather to inform trends and behaviours at a particular station. The library has 4 of them :

- [Time to Peak](#time-to-peak)
- [Time to Centre of Mass](#time-to-centre-of-mass)
- [Spring Pulse Onset Delay](#spring-pulse-onset-delay)
- [Slope of the Flow Duration Curve](#flow-duration-curve-slope)

#### Time to Peak
This helps to show how long it takes on average to get to the highest streamflow each year. An example is shown below:

In [5]:
# The Time to Peak for the simulated data will look like 
print(metrics.time_to_peak(df=DATAFRAMES['DF_SIMULATED']))

# The time to peak for the observed data looks like:-
print(metrics.time_to_peak(df=DATAFRAMES['DF_OBSERVED']))

[171.0, 177.0, 176.0, 169.0, 169.0, 174.0, 166.0, 157.0, 157.0, 168.0, 179.0, 162.0, 172.0, 171.0, 171.0, 175.0, 168.0, 188.0, 187.0, 184.0, 187.0, 174.0, 173.0, 207.0, 175.0, 184.0, 147.0, 148.0, 151.0, 184.0, 141.0, 145.0, 160.0, 176.0, 177.0, 165.0, 208.0, 177.0, 147.0, 158.0]
[157.0, 157.0, 158.0, 159.0, 160.0, 172.0, 175.0, 166.0, 165.0, 173.0, 189.0, 169.0, 164.0, 169.0, 167.0, 167.0, 171.0, 163.0, 169.0, 159.0, 156.0, 184.0, 178.0, 179.0, 184.0, 174.0, 119.0, 115.0, 121.0, 172.0, 128.0, 123.0, 134.0, 123.0, 158.0, 111.0, 150.0, 152.0, 161.0, 139.0]


As you can see, at the first station, on average, over the years, the highest predicted streamflow value will usually occur after 170 days - somewhere in the third week of June. For the second station on average, over the years, the highest predicted streamflow value usually occur after 177 days - somewhere in the final week of June. 
As you can see, you are able to observe and notice trends with the data at specific stations.

#### Time to Centre of Mass
This helps to show how long it takes on average to obtain 50% of the streamflow each year. An example is shown below:

In [6]:
# The Time to Centre of Mass for the simulated data will look like 
print(metrics.time_to_centre_of_mass(df=DATAFRAMES['DF_SIMULATED']))

# The time to Centre of Mass for the observed data looks like:-
print(metrics.time_to_centre_of_mass(df=DATAFRAMES['DF_OBSERVED']))

[185.0, 166.0, 188.0, 183.0, 186.0, 186.0, 190.0, 183.0, 181.0, 158.0, 175.0, 185.0, 170.0, 182.0, 182.0, 184.0, 187.0, 176.0, 190.0, 190.0, 193.0, 187.0, 193.0, 205.0, 190.0, 190.0, 155.0, 150.0, 163.0, 189.0, 163.0, 167.0, 172.0, 173.0, 188.0, 170.0, 193.0, 189.0, 156.0, 185.0]
[177.0, 179.0, 183.0, 178.0, 180.0, 178.0, 199.0, 194.0, 186.0, 172.0, 204.0, 186.0, 177.0, 190.0, 187.0, 177.0, 194.0, 172.0, 182.0, 177.0, 180.0, 195.0, 200.0, 184.0, 202.0, 183.0, 150.0, 132.0, 138.0, 183.0, 154.0, 149.0, 146.0, 157.0, 181.0, 150.0, 187.0, 173.0, 179.0, 179.0]


As you can see, at the first station, on average, over the years, 50% of the total volume of streamflow each year will usually have occured by 177 days - somewhere in the final week of June and for the second stations, after 179 days - Right at the end of June. 

#### Spring Pulse Onset Delay
This is used to determine what day snowmelt starts. An example is shown below:

In [7]:
# The Spring Pulse Onset for the simulated data will look like 
print(metrics.SpringPulseOnset(df=DATAFRAMES['DF_SIMULATED']))

# The Spring Pulse Onset for the observed data looks like:-
print(metrics.SpringPulseOnset(df=DATAFRAMES['DF_OBSERVED']))

[128.0, 127.5, 115.0, 115.05882352941177, 127.0, 127.02941176470588, 119.0, 118.79411764705883, 122.0, 121.5, 124.0, 124.44117647058823, 140.0, 139.7941176470588, 128.0, 127.55882352941177, 126.0, 126.0, 296.0, 296.5, 113.0, 113.41176470588235, 136.0, 135.61764705882354, 110.0, 110.11764705882354, 124.0, 124.02941176470588, 126.0, 126.05882352941177, 129.0, 128.52941176470588, 128.0, 128.1764705882353, 123.0, 122.79411764705883, 143.0, 142.55882352941177, 145.0, 145.11764705882354, 145.0, 145.1764705882353, 123.0, 122.52941176470588, 120.0, 119.88235294117646, 198.0, 197.88235294117646, 137.0, 137.47058823529412, 121.0, 121.26470588235294, 109.0, 109.1470588235294, 98.9, 98.88235294117646, 113.0, 113.05882352941177, 121.0, 120.76470588235294, 120.0, 120.17647058823529, 116.0, 116.23529411764706, 112.0, 112.08823529411765, 114.0, 113.52941176470588, 117.0, 117.20588235294117, 134.0, 134.41176470588235, 185.0, 184.7941176470588, 146.0, 146.2941176470588, 103.0, 103.44117647058823, 111.0,

C:\Users\udenzeU\AppData\Local\anaconda3\envs\postprocessing\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\udenzeU\AppData\Local\anaconda3\envs\postprocessing\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[113.0, 112.53125, nan, nan, 101.0, 101.09677419354838, 114.0, 114.0, 115.0, 114.96969696969697, 124.0, 124.23529411764706, 142.0, 141.8, 136.0, 136.44117647058823, 138.0, 138.3235294117647, 296.0, 296.4516129032258, 172.0, 171.5, 139.0, 138.7941176470588, 118.0, 118.29411764705883, nan, nan, nan, nan, 156.0, 155.7941176470588, 135.0, 134.76470588235293, 101.0, 100.87878787878788, 112.0, 112.08823529411765, 103.0, 102.79411764705883, 107.0, 107.17647058823529, 138.0, 138.14705882352942, nan, nan, 140.0, 139.5, nan, nan, 106.0, 105.58823529411765, nan, nan, nan, nan, nan, nan, 101.0, 101.0909090909091, 93.5, 93.45454545454545, nan, nan, nan, nan, 89.5, 89.5, 98.4, 98.3529411764706, 69.3, 69.3125, 206.0, 205.55172413793105, 126.0, 126.02941176470588, 108.0, 107.82352941176471, 99.4, 99.4375]


This shows us that at the first station, on average, over the years, snowmelt is predicted to begin 127 days into the year - somewhere in the First week of May. For the second station on average, over the years, snowmelt is predicted to begin 127.44 days into the year - somewhere in the First week of May as well

#### Flow Duration Curve Slope
This is used to calculate the slope of the flow duration curve. An example is shown below:

In [8]:
# The Fliw Duration Curve for the Simulated Data will look like 
print(metrics.slope_fdc(df=DATAFRAMES['DF_SIMULATED']))

# You can also specify which percentile to pick values from 
print(metrics.slope_fdc(df=DATAFRAMES['DF_SIMULATED'], percentiles=[25, 77]))

[3.1566, 2.3474, 2.1153, 4.3959, 4.5556, 3.3224, 7.9926, 7.0947, 1.5771, 4.104, 7.04, 2.3884, 5.6952, 2.7031, 2.6837, 2.6551, 5.6625, 5.5075, 1.1545, 1.3746, 1.589, 6.1219, 0.7689, 0.71, 6.3549, 1.0757, 4.4023, 3.1112, 4.9827, 1.279, 5.9984, 4.5465, 3.5594, 3.4349, 1.6837, 5.7192, 1.0864, 0.7688, 5.2666, 1.2176]
[2.6734, 2.9447, 2.1918, 3.6673, 3.8495, 3.1524, 7.4472, 6.2342, 2.0077, 5.0327, 6.8506, 2.4762, 6.1256, 2.8523, 2.6235, 2.6343, 5.2397, 5.8473, 1.5813, 1.838, 2.0295, 5.7037, 1.3392, 0.6891, 5.7742, 1.5099, 4.4881, 3.1196, 5.578, 1.613, 6.0078, 5.0091, 3.7477, 3.4018, 1.8384, 5.749, 1.1505, 0.9766, 6.2386, 1.3281]


### Comparison Metrics

These are the metrics that are used to compare the simulated and observed data. They work to show accurately we are able to predict the streamflow values using the models. Every other metric is a comparison metric. They are shown below:

- [Mean Square Error](#mean-square-error)
- [Root Mean Square Error](#root-mean-square-error)
- [Mean Average Error](#mean-average-error)
- [Nash-Sutcliffe Efficiency](#nash-sutcliffe-efficiency)
- [Kling-Gupta Efficiency](#kling-gupta-efficiency)
- [Percentage Bias](#percentage-bias)

#### Mean Square Error


In [9]:
# Mean square error for the data we were given
print(metrics.mse(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

[1299.0, 780.6, 17.22, 5596.0, 5723.0, 16010.0, 85.47, 573.6, 1823.0, 45.17, 88.77, 1894.0, 522.4, 4731.0, 6572.0, 4964.0, 622.2, 123.1, 1417.0, 2296.0, 3495.0, 792.0, 11070.0, 1875.0, 1284.0, 13940.0, 54.71, 8.839, 55.62, 17770.0, 21.09, 93.04, 178.0, 191.9, 20800.0, 35.14, 25030.0, 74190.0, 2432.0, 142400.0]


#### Root Mean Square Error

In [10]:
# Root Mean square error for the data we were given
print(metrics.rmse(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

[36.05, 27.94, 4.15, 74.8, 75.65, 126.5, 9.245, 23.95, 42.69, 6.721, 9.422, 43.52, 22.86, 68.78, 81.07, 70.46, 24.94, 11.1, 37.64, 47.92, 59.12, 28.14, 105.2, 43.3, 35.84, 118.1, 7.397, 2.973, 7.458, 133.3, 4.592, 9.646, 13.34, 13.85, 144.2, 5.928, 158.2, 272.4, 49.32, 377.4]


#### Mean Average Error

In [11]:
# Mean Average error for the data we were given
print(metrics.mae(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

[16.75, 6.936, 1.236, 39.51, 41.5, 80.41, 6.762, 17.81, 36.39, 5.119, 4.13, 36.09, 10.41, 46.12, 55.54, 46.46, 12.17, 3.986, 18.83, 22.68, 27.06, 12.24, 59.72, 33.07, 25.69, 68.21, 3.213, 1.1, 2.087, 74.06, 1.524, 4.569, 5.795, 5.024, 78.12, 1.567, 89.51, 172.8, 18.74, 218.8]


#### Nash-Sutcliffe Efficiency

In [12]:
# Nash-Sutcliffe Efficiency for the data we were given
print(metrics.nse(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

[0.5166, -1.675, -1.982, 0.6125, 0.6057, 0.6167, 0.3663, 0.6654, 0.3366, -0.3894, -2.133, 0.4652, 0.711, 0.6899, 0.5733, 0.4938, 0.4015, -0.1562, 0.5873, 0.4797, 0.3275, -0.9159, -0.7521, -1.618, 0.1876, 0.3868, -0.8777, -0.07323, -2.481, 0.3183, 0.3262, -0.04744, 0.1615, -0.3545, 0.3504, -0.2135, 0.06404, 0.0938, -4.977, -0.1328]


##### Logarithm of the Nash-Sutcliffe Efficiency

In [13]:
# Logarithm of the Nash-Sutcliffe Efficiency for the data we were given
print(metrics.lognse(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

[-0.2511, -0.1692, -0.0115, 0.1126, 0.1109, -0.212, -0.4735, -1.649, -1.849, 0.03756, -24.22, -1.991, -1.017, -0.03542, 0.03157, -0.5146, -0.01198, -1.623, 0.02779, 0.1001, 0.2395, -1.441, -1.804, -7.492, -0.8192, -0.9654, -0.4353, -0.146, 0.04298, -0.3772, -1.155, 0.05425, 0.1638, 0.4553, 0.1876, -8.318, 0.2288, 0.1655, -0.6624, 0.2843]


#### Kling-Gupta Efficiency

In [14]:
# Kling-Gupta Efficiency for the data we were given
print(metrics.kge(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

[0.5094, -0.1113, -0.01851, 0.764, 0.7467, 0.7972, 0.5978, 0.5852, 0.5825, 0.09986, -0.2227, 0.6184, 0.724, 0.7723, 0.6945, 0.7018, 0.5146, 0.3796, 0.7594, 0.6983, 0.6529, -0.01509, 0.08618, 0.03342, 0.4338, 0.6025, 0.2302, 0.4502, -0.1606, 0.6241, 0.2493, 0.2491, 0.279, 0.3962, 0.6373, -0.08113, 0.5701, 0.5738, -0.81, 0.4515]


##### Modified Kling Gupta efficiency
This is different from the regular kge in that this uses the coefficient of Variation as its bias term (i.e., std/mean) as opposed to just the mean

In [15]:
# Kling-Gupta Efficiency for the data we were given
print(metrics.kge_2012(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

[0.5606, 0.08006, -0.2077, 0.7312, 0.712, 0.8105, 0.3187, 0.3659, 0.1129, 0.1105, -0.1818, 0.2103, 0.5842, 0.7162, 0.7484, 0.6213, 0.6522, -0.1999, 0.6652, 0.6515, 0.5855, 0.1769, -0.07924, 0.00923, 0.2124, 0.4171, -0.1117, 0.494, -0.2118, 0.4499, 0.3274, -0.01532, 0.05332, 0.2218, 0.5214, -0.1135, 0.5859, 0.6142, -0.07765, 0.4249]


#### Percentage Bias

In [16]:
# Percentage Bias for the data we were given
print(metrics.bias(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

[-34.16, 11.5, -13.79, 13.5, 16.58, 2.373, -20.24, -38.9, -35.18, -48.37, 2.078, -33.57, -23.84, -5.14, 19.41, -7.604, 11.77, -36.0, -10.93, -15.98, -14.41, 11.27, -8.41, -36.1, -13.27, -13.65, -25.84, 28.26, -3.039, -14.83, -49.71, -49.13, -51.86, -19.74, -10.12, -57.19, 7.11, 5.836, 56.11, -2.214]


Now that we have seen individual metrics, we also have the ability to calculate a list of metrics using our **calculate_all_metrics** or **calculate_metrics(list of merics)**. These are shown below:

In [17]:
metrices = ["MSE", "RMSE", "MAE", "NSE", "NegNSE"]
for key, value in metrics.calculate_metrics(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED'],
                                            metrices=metrices).items():
    print(f"{key}:\n {value}")

MSE:
 [1299.0, 780.6, 17.22, 5596.0, 5723.0, 16010.0, 85.47, 573.6, 1823.0, 45.17, 88.77, 1894.0, 522.4, 4731.0, 6572.0, 4964.0, 622.2, 123.1, 1417.0, 2296.0, 3495.0, 792.0, 11070.0, 1875.0, 1284.0, 13940.0, 54.71, 8.839, 55.62, 17770.0, 21.09, 93.04, 178.0, 191.9, 20800.0, 35.14, 25030.0, 74190.0, 2432.0, 142400.0]
RMSE:
 [36.05, 27.94, 4.15, 74.8, 75.65, 126.5, 9.245, 23.95, 42.69, 6.721, 9.422, 43.52, 22.86, 68.78, 81.07, 70.46, 24.94, 11.1, 37.64, 47.92, 59.12, 28.14, 105.2, 43.3, 35.84, 118.1, 7.397, 2.973, 7.458, 133.3, 4.592, 9.646, 13.34, 13.85, 144.2, 5.928, 158.2, 272.4, 49.32, 377.4]
MAE:
 [16.75, 6.936, 1.236, 39.51, 41.5, 80.41, 6.762, 17.81, 36.39, 5.119, 4.13, 36.09, 10.41, 46.12, 55.54, 46.46, 12.17, 3.986, 18.83, 22.68, 27.06, 12.24, 59.72, 33.07, 25.69, 68.21, 3.213, 1.1, 2.087, 74.06, 1.524, 4.569, 5.795, 5.024, 78.12, 1.567, 89.51, 172.8, 18.74, 218.8]
NSE:
 [0.5166, -1.675, -1.982, 0.6125, 0.6057, 0.6167, 0.3663, 0.6654, 0.3366, -0.3894, -2.133, 0.4652, 0.711, 0.68

We are also able to save these metrics as text files and csv files by specifying the **format** parameter and even the **out** parameter to specify a name to save it as.

In [20]:
# metrics.calculate_metrics(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED'], metrices=metrices,
#                          format='txt', out='metrics'
#                          )